# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

sentence tokenizer (?) and classification

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /home/michelle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/michelle/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [5]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [6]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31524    0 31524    0     0  22184      0 --:--:--  0:00:01 --:--:-- 22200


In [7]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70549    0 70549    0     0  50296      0 --:--:--  0:00:01 --:--:-- 50320


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 11, relationships: 35)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 11, relationships: 35)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

I found a reference here: https://docs.ragas.io/en/stable/concepts/test_data_generation/rag/

A single-hop query is a question that requires retrieving information from a single document or source to provide a relevant answer. It involves only one step to arrive at the answer.  ** In this example 50% of the queries are single-hop.

A multi-hop query involves multiple steps of reasoning, requiring information from two or more sources (in context). The system must retrieve information from various documents and connect the dots (reason) to generate an accurate answer.

- the abstract query requires the retrieval of multiple pieces of information over time and across different sources to form a broad, interpretive response.  ** In this example 25% of the queries are multi-hop abstract.

- the specific query focuses on clear, fact-based retrieval.  ** In this example 25% of the queries are multi-hop specific.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is Claude in AI?,[The year of slop Synthetic training data work...,Claude is a model mentioned in the context as ...,single_hop_specifc_query_synthesizer
1,How does GitHub's announcement of GitHub Spark...,[a similar way to ChatGPT now. Amazon also pre...,GitHub announced their version of prompt-drive...,single_hop_specifc_query_synthesizer
2,How does Google Search's performance relate to...,[run into the same roadblock: how good is a tr...,The context discusses how AI systems like Goog...,single_hop_specifc_query_synthesizer
3,What is DeepSeek?,[Nothing yet from Anthropic or Meta but I woul...,DeepSeek v3 is a huge 685B parameter model—one...,single_hop_specifc_query_synthesizer
4,What is GPT-4o,[Phi series of models has consistently emphasi...,GPT-4o is a model that can run web searches an...,single_hop_specifc_query_synthesizer
5,How do synthetic training data and its advanta...,[<1-hop>\n\nPhi series of models has consisten...,Synthetic training data plays a crucial role i...,multi_hop_abstract_query_synthesizer
6,Considering the significant advancements in AI...,[<1-hop>\n\nThe year of slop Synthetic trainin...,"In 2024, AI research and development have made...",multi_hop_abstract_query_synthesizer
7,How does the environmental impact of AI traini...,[<1-hop>\n\nNothing yet from Anthropic or Meta...,The context indicates that recent AI models li...,multi_hop_abstract_query_synthesizer
8,Considering the detailed explanations of how L...,[<1-hop>\n\nthe document includes some of the ...,The recent advancements in multimodal vision a...,multi_hop_specific_query_synthesizer
9,How does the development of large language mod...,[<1-hop>\n\nNothing yet from Anthropic or Meta...,"DeepSeek v3, trained in China for less than $6...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [18]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [19]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Whaat is the relashunship betwen Netflix and t...,[The year of slop Synthetic training data work...,The context mentions Netflix as part of a prom...,single_hop_specifc_query_synthesizer
1,what is chatgpt,[a similar way to ChatGPT now. Amazon also pre...,ChatGPT is a feature that now provides the opt...,single_hop_specifc_query_synthesizer
2,How good is Claude in distinguishing truth fro...,[run into the same roadblock: how good is a tr...,The context highlights that models like Claude...,single_hop_specifc_query_synthesizer
3,What is Meta's recent contribution to large la...,[Nothing yet from Anthropic or Meta but I woul...,Meta published a relevant paper titled 'Traini...,single_hop_specifc_query_synthesizer
4,H0w do long inputs and complex tasks help AI t...,[<1-hop>\n\nThe year of slop Synthetic trainin...,"Long inputs, such as models accepting up to 2 ...",multi_hop_abstract_query_synthesizer
5,How do inference-scaling models like OpenAI’s ...,[<1-hop>\n\nrun into the same roadblock: how g...,Inference-scaling models such as OpenAI’s o1 a...,multi_hop_abstract_query_synthesizer
6,How do recent advancements in large language m...,[<1-hop>\n\nThe year of slop Synthetic trainin...,Recent advancements in large language models (...,multi_hop_abstract_query_synthesizer
7,How do advancements in LLMs like GPT-4-0314 an...,[<1-hop>\n\nThe year of slop Synthetic trainin...,"The context shows that in 2024, models like GP...",multi_hop_abstract_query_synthesizer
8,How does the Llama model's emphasis on synthet...,[<1-hop>\n\nthe document includes some of the ...,The context emphasizes that the Phi series of ...,multi_hop_specific_query_synthesizer
9,wha is GPT-4o and GPT-4 and how are they related?,[<1-hop>\n\nThe ethics of this space remain di...,GPT-4o is a version of GPT-4 that includes add...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years! r4"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

In [25]:
docs

[Document(metadata={'source': 'data/2024_llms.html'}, page_content='Simon Willison’s Weblog\n\nSubscribe\n\nThings we learned about LLMs in 2024\n\n31st December 2024\n\nA lot has happened in the world of Large Language Models over the course of 2024. Here’s a review of things we figured out about the field in the past twelve months, plus my attempt at identifying key themes and pivotal moments.\n\nThis is a sequel to my review of 2023.\n\nIn this article:\n\nThe GPT-4 barrier was comprehensively broken\n\nSome of those GPT-4 models run on my laptop\n\nLLM prices crashed, thanks to competition and increased efficiency\n\nMultimodal vision is common, audio and video are starting to emerge\n\nVoice and live camera mode are science fiction come to life\n\nPrompt driven app generation is a commodity already\n\nUniversal access to the best models lasted for just a few short months\n\n“Agents” still haven’t really happened yet\n\nEvals really matter\n\nApple Intelligence is bad, Apple’s MLX 

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [27]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [28]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

In [29]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [30]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [31]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [32]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [33]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an infuriatingly vague term with no single, clear, and widely understood meaning. Broadly, it seems to converge on AI systems that can go away and act on your behalf. There are two main interpretations:\n\n1. AI agents as entities that act on your behalf, similar to a travel agent model.\n2. Large Language Models (LLMs) equipped with access to tools, which they can use iteratively to solve problems.\n\nHowever, despite much discussion, few if any true AI agents are running in production yet. The concept of agents is also tied to challenges such as gullibility, where AI systems struggle to distinguish truth from fiction, limiting their practical autonomy and utility. Some believe that fully solving these issues may require achieving Artificial General Intelligence (AGI).\n\nIn summary, "agents" refer to AI systems intended to autonomously act on behalf of users, but the term remains unclear and the technology is still mostly in prototype or co

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [34]:
eval_llm = ChatOpenAI(model="gpt-4.1-mini")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [35]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: question answer evaluator evaluates the accuracy of the model answers compared to the reference answers, did it correctly anser the question
- `labeled_helpfulness_evaluator`: evaluates the response by using the llm response and the reference answer to determine helpfulness, how helpful the response may be to the user
	* compare the strings for answers and determine if the submission was helpful
	* prediction output = is what chain gives us
	* reference is our answer = reference answer created by ragas
	* input the question
- `dope_or_nope_evaluator`: evaluates the response to determine how dope, lit or cool the llm response is

## LangSmith Evaluation

In [36]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'virtual-salt-70' at:
https://smith.langchain.com/o/188227a9-0311-509f-a89d-cb574a93bee1/datasets/d6b16160-3f9b-4301-843d-dc6a152307e0/compare?selectedSessions=f64c8963-f728-4c43-b805-55f84be832fd




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,how ChatGPT and ChatGPT voice mode are related...,Based on the context provided:\n\nChatGPT voic...,None,ChatGPT now has a voice mode that lets you sha...,1,1,1,6.587699,2ace7436-7359-417b-ac5e-051b4eece455,0eabd688-d637-4fd5-ab26-0a67f24f2311
1,Llama 3.3 and Llama 3.1 which one better for l...,I don't know.,None,Llama 3.3 is a newer model that is part of Met...,0,0,0,0.722359,2a2c8487-d1d8-4f7d-afd5-12dde4f41db0,2dded5c3-781e-460d-a3c5-27d638ba7072
2,wha is GPT-4o and GPT-4 and how are they related?,Based on the context:\n\nGPT-4 is OpenAI's ori...,None,GPT-4o is a version of GPT-4 that includes add...,0,0,1,2.184441,ec8f61cf-a6a7-4d8c-b511-a8260a57c3bb,a2653909-3e01-43ea-a9f5-31229ab159be
3,How does the Llama model's emphasis on synthet...,I don't know.,None,The context emphasizes that the Phi series of ...,0,0,0,0.861350,e4196c57-70e8-46c1-be75-f45830c05b95,fe013c4f-f643-4b7d-926e-b5d4385b2e9a
4,How do advancements in LLMs like GPT-4-0314 an...,Advancements in LLMs such as GPT-4-0314 and mo...,None,"The context shows that in 2024, models like GP...",1,0,1,2.404002,5e32f049-501d-4312-a379-7c99b1d31882,63de8c25-7e88-4f7c-bf52-f9ee997f4b1b
5,How do recent advancements in large language m...,Recent advancements in large language models (...,None,Recent advancements in large language models (...,1,1,1,6.966426,a88bf30d-0a7c-41c5-97cd-3076f2fa6a7f,d8c5f996-10ab-4494-bf39-7a0451725423
6,How do inference-scaling models like OpenAI’s ...,I don't know.,None,Inference-scaling models such as OpenAI’s o1 a...,0,0,0,0.879519,f928b193-0163-4af5-984e-cba39bd20b55,81f2475c-3f4e-49a8-8d69-bd0c5799dadb
7,H0w do long inputs and complex tasks help AI t...,"Based on the provided context, long inputs and...",None,"Long inputs, such as models accepting up to 2 ...",1,1,0,4.174207,427e9015-2c7c-4e05-9ed3-8c89d5ecc99f,a3a71145-c4a6-419a-8cbe-0761b8a99436
8,What is Meta's recent contribution to large la...,Meta's recent contributions to large language ...,None,Meta published a relevant paper titled 'Traini...,0,1,1,2.791713,48c00c07-fe13-4d23-8de4-3908b4f3fc06,c803042d-5158-4faf-8094-8e763b23bbd3
9,How good is Claude in distinguishing truth fro...,"Based on the provided context, Claude, includi...",None,The context highlights that models like Claude...,1,1,1,3.019526,7d310ead-be6f-4cc6-a6ef-23f26fea961b,4a4ebcdc-5ae4-41aa-af96-7f2663538f4c


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [37]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [38]:
rag_documents = docs

In [39]:
docs

[Document(metadata={'source': 'data/2024_llms.html'}, page_content='Simon Willison’s Weblog\n\nSubscribe\n\nThings we learned about LLMs in 2024\n\n31st December 2024\n\nA lot has happened in the world of Large Language Models over the course of 2024. Here’s a review of things we figured out about the field in the past twelve months, plus my attempt at identifying key themes and pivotal moments.\n\nThis is a sequel to my review of 2023.\n\nIn this article:\n\nThe GPT-4 barrier was comprehensively broken\n\nSome of those GPT-4 models run on my laptop\n\nLLM prices crashed, thanks to competition and increased efficiency\n\nMultimodal vision is common, audio and video are starting to emerge\n\nVoice and live camera mode are science fiction come to life\n\nPrompt driven app generation is a commodity already\n\nUniversal access to the best models lasted for just a few short months\n\n“Agents” still haven’t really happened yet\n\nEvals really matter\n\nApple Intelligence is bad, Apple’s MLX 

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

smaller chunks would perform faster and prob use less tokens, but maybe lose some meaning

semantic chunking is much more accurate but potentially less performant than smaller chunking strategies like word or sentence


In [41]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

each embedding model has it's own performance characteristics, quality and speed can vary based on the model.  some are better suited to specific tasks, ie sentiment.

In [42]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [43]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [44]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [45]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, here’s the lowdown on Agents, straight from the street of AI talk:\n\n“Agents” is one of those buzzwords that’s got more meanings than a chameleon’s closet. At its core, people toss it around to mean AI systems that hustle on your behalf—like a travel agent booking your trips or an assistant running errands digitally. But here’s the kicker: nobody’s really nailed down a solid, universal definition. Some folks see agents as LLMs equipped with tools they can loop through to solve problems, others think of them as fully autonomous doers. \n\nThe real catch? Agents are stuck in “coming soon” mode because of the big baddie called gullibility—LLMs believe *any* spiel you feed them, which makes trusting them with decisions pretty dicey. Without solving that, agents can’t really shine for real-world use. So yeah, they’re hyped, kind of conceptually lit, but still mostly prototypes trying to get out the door.\n\nIn short: Agents are AI homies meant to act for you, but right now they’r

Finally, we can evaluate the new chain on the same test set!

In [ ]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'large-salt-64' at:
https://smith.langchain.com/o/188227a9-0311-509f-a89d-cb574a93bee1/datasets/d6b16160-3f9b-4301-843d-dc6a152307e0/compare?selectedSessions=0340d921-bdfc-4231-a067-9872569d9a7a




0it [00:00, ?it/s]

#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

